In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import numpy as np
import thuner.data as data
import thuner.option as option
import thuner.analyze as analyze
import thuner.parallel as parallel
import thuner.visualize as visualize

import thuner.log as log

notebook_name = "gridrad_demo.ipynb"


Welcome to the Thunderstorm Event Reconnaissance (THUNER) package v0.0.6! This 
is a placeholder version of the package and is not yet functional. Please visit 
github.com/THUNER-project/THUNER to report issues or contribute.

THUNER is a flexible toolkit for performing multi-feature detection, tracking, tagging
and analysis of events within meteorological datasets. The intended application is to
convective weather events. For examples and instructions, see 
github.com/THUNER-project/THUNER. If you use this package in your research, consider 
citing the following papers;

Short et al. (2023), MWR, doi: 10.1175/MWR-D-22-0146.1
Raut et al. (2021), JAMC, doi: 10.1175/JAMC-D-20-0119.1
Fridlind et al. (2019), AMT, doi: 10.5194/amt-12-2979-2019
Dixon and Wiener (1993), JTECH, doi: 10.1175/1520-0426(1993)010<0785:TTITAA>2.0.CO;2
Leese et al. (1971), JAMC, doi: 10.1175/1520-0450(1971)010<0118:AATFOC>2.0.CO;2



In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THUNER_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)
# start = "2010-01-20T19:20:00"
# end = "2010-01-21T00:30:00"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/gridrad_demo"

In [3]:
# if output_parent.exists():
# shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create and save the dataset options
times_dict = {"start": start, "end": end}
gridrad_dict = {"event_start": event_start}
gridrad_options = data.gridrad.GridRadSevereOptions(**times_dict, **gridrad_dict)
era5_dict = {"latitude_range": [27, 39], "longitude_range": [-102, -89]}
era5_pl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
datasets = [gridrad_options, era5_pl_options, era5_sl_options]
data_options = option.data.DataOptions(datasets=datasets)
data_options.to_yaml(options_directory / "data.yml")

# Create and save the grid_options dictionary
kwargs = {"name": "geographic", "regrid": False, "altitude_spacing": None}
kwargs.update({"geographic_spacing": None})
grid_options = option.grid.GridOptions(**kwargs)
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options dictionary
track_options = option.default.track(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
# track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
# track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

visualize_options = None

2025-02-06 17:32:33,864 - thuner.data.gridrad - INFO - Generating era5 filepaths.
2025-02-06 17:32:33,868 - thuner.data.era5 - INFO - Generating era5 filepaths.
2025-02-06 17:32:33,870 - thuner.data.era5 - INFO - Generating era5 filepaths.
2025-02-06 17:32:33,899 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-02-06 17:32:33,901 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.


In [4]:
times = data.utils.generate_times(data_options.dataset_by_name("gridrad"))
args = [times, data_options, grid_options, track_options, visualize_options]
parallel.track(*args, output_directory=output_parent)

2025-02-06 17:32:45,852 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_0.
2025-02-06 17:32:45,852 - thuner.track - INFO - Beginning simultaneous tracking.
2025-02-06 17:32:45,857 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_1.
2025-02-06 17:32:45,857 - thuner.track - INFO - Beginning simultaneous tracking.
2025-02-06 17:32:45,924 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_2.
2025-02-06 17:32:45,925 - thuner.track - INFO - Beginning simultaneous tracking.
2025-02-06 17:32:46,036 - thuner.track - INFO - Beginning thuner run. Saving output to /home/ewan/THUNER_output/runs/gridrad_demo/interval_3.
2025-02-06 17:32:46,036 - thuner.track - INFO - Beginning simultaneous tracking.
2025-02-06 17:32:46,418 - thuner.track - INFO - Processing 2010-01-20T19:00:00.
2025-02-06 17:32:46,419 - th

In [5]:
analysis_options = analyze.mcs.AnalysisOptions()
analyze.mcs.process_velocities(output_parent, profile_dataset=None)
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent, analysis_options)

2025-02-06 18:21:14,508 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-02-06 18:21:14,509 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.
2025-02-06 18:21:14,706 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-02-06 18:21:14,706 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.


In [6]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
kwargs = {"style": "presentation", "attributes": ["velocity", "offset"]}
figure_options = visualize.option.horizontal_attribute_options(figure_name, **kwargs)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

2025-02-06 16:54:25,952 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-02-06 16:54:25,953 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.


2025-02-06 16:54:26,447 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:00:00.000000000.
2025-02-06 16:54:28,033 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-02-06 16:54:28,034 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.
2025-02-06 16:54:29,011 - thuner.visualize.attribute - INFO - Saving mcs_gridrad_20100120 figure for 2010-01-20T18:00:00.000000000.



Welcome to the Thunderstorm Event Reconnaissance (THUNER) package v0.0.6! This 
is a placeholder version of the package and is not yet functional. Please visit 
github.com/THUNER-project/THUNER to report issues or contribute.

THUNER is a flexible toolkit for performing multi-feature detection, tracking, tagging
and analysis of events within meteorological datasets. The intended application is to
convective weather events. For examples and instructions, see 
github.com/THUNER-project/THUNER. If you use this package in your research, consider 
citing the following papers;

Short et al. (2023), MWR, doi: 10.1175/MWR-D-22-0146.1
Raut et al. (2021), JAMC, doi: 10.1175/JAMC-D-20-0119.1
Fridlind et al. (2019), AMT, doi: 10.5194/amt-12-2979-2019
Dixon and Wiener (1993), JTECH, doi: 10.1175/1520-0426(1993)010<0785:TTITAA>2.0.CO;2
Leese et al. (1971), JAMC, doi: 10.1175/1520-0450(1971)010<0118:AATFOC>2.0.CO;2


Welcome to the Thunderstorm Event Reconnaissance (THUNER) package v0.0.6! This 
is 

2025-02-06 16:54:36,923 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:10:00.000000000.
2025-02-06 16:54:36,969 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:20:00.000000000.
2025-02-06 16:54:37,023 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:30:00.000000000.
2025-02-06 16:54:39,012 - thuner.visualize.attribute - INFO - Visualizing MCS at time 2010-01-20T18:40:00.000000000.
2025-02-06 16:54:39,947 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-02-06 16:54:39,947 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.
2025-02-06 16:54:40,008 - thuner.option.grid - WARNING - altitude_spacing not specified. Will attempt to infer from input.
2025-02-06 16:54:40,009 - thuner.option.grid - WARNING - shape not specified. Will attempt to infer from input.
2025-02-06 16:54:40,067 - thuner.option.grid - WARNING - altit